In [ ]:
from google.cloud import pubsub_v1
from google.oauth2 import service_account
import json
from pprint import pprint

service_account_path = "secrets/service_account.json"
project_id = "nytint-stg"
subscription_id = "vi_workflow_topic-sub"

credentials = service_account.Credentials.from_service_account_file(service_account_path)

subscriber = pubsub_v1.SubscriberClient(credentials=credentials)
subscription_path = subscriber.subscription_path(project_id, subscription_id)

def callback(message):
    print(f"Received message")
    message_data = message.data.decode('utf-8')
    message_dict = json.loads(message_data)

    pprint(message_dict)
    
    message.ack()


streaming_pull_future = subscriber.subscribe(subscription_path, callback=callback)
print(f"Listening for messages on {subscription_path}...\n")

try:
    streaming_pull_future.result()  # Block and wait for messages
except KeyboardInterrupt:
    streaming_pull_future.cancel()  # Trigger the shutdown
    streaming_pull_future.result()  # Block until the shutdown is complete
